# Overview

This notebook is a simple overview of how to use the open AI APIs to build some simple applications. Currently there are two:
1) A simple chatbot app; "AI therapist"
2) An inexpensive text-to-image generator.

You can get an OpenAI API key for free for a trial period. Even after the trial finishes, you may decide to keep using it as it is a relative bargain. Consider:
1) OpenAI plus is $20/month, and it is required to use DALL-E, or to get more access to the latest chatGPT models
2) Meanwhile, at the API level, you can generate images for just a few cents per image, and perform chat sessions for less than a penny per session.

# Prereqs

To get things set up, you'll need to set up a google colab account to run this notebook (and then modify/personalize it). You will also need to get an OpenAI API key.

## Sign up for a Google Colab account
1. Create a [Google colab](https://colab.research.google.com) account if you don't already have one. Colab was specifically built for creating and sharing interactive python data science code, and it's free tier is quite feature rich.
2. Clone this notebook like so:
    1. Go to "Open Notebook" (it should have popped up by default, but if not you can find it under the "File" menu)
    2. Select "GitHub"
    3. Enter the following path: https://github.com/marcshepard/teaching-data-science/blob/main/openai.ipynb
    4. Click the search icon

## Obtain an OpenAI API key
Go to the [Open AI developer](https://platform.openai.com) site.
Click on "API keys" on the left tab.
Create a new "project API SECRET key".
1. *Keep it somewhere* - once you move off that screen you can never see the secret again. They won't let you view it later after you have created it. 
2. *Keep it secret* - don't put it on any public sites or anywhere others can get at it.
Here's an idea: Store it as a Google Colab secret (next section), as Colab will keep it secret and allows you to view it later if you need.

## Copy your API key to Colab as a secret
Meanwhile in Colab, click on the key icon on the left nav.
Click "Add new secret":
* Give it a name of "OPENAI_KEY"
* Paste the value you got from OpenAI.
* Toggle the "Notebook access" button

OK; you are now ready to start building things on their APIs. But before you do, let's get a feel for how they work...

# Play with the OpenAI platform console
The [playground](https://platform.openai.com/playground) is a great place to get a feel for how the APIs work.

## Chat
Let's start in the Chat tab:

**System** The System message guides how the chatbot responds. Try entering the following text (or something else if you prefer):
<p style="margin:10px">You are an AI therapist with great empathy and listening skills, specializing in CBT, DBT, and committed to helping your patients understand and work through any psychological challenges they may have</p>

At the bottom of the screen, enter some message, like "hi, doc" and then click "run". Have a quick chat. Then take a few minutes to play around with other configurations:
* Different System contexts
* Different values of the sliders on the right:
    * Temperature - higher = more creative/random responses; lower = more deterministic
    * Top P - similar effect to temperature
    * Frequency - higher = less repetition in the output (e.g, for creative writing)
    * Presense - higher = increase likelyhood to introduce new topics (e.g., for brainstorming)


## Assistant
Assistant is a newer chat model that adds:
1. *Files* can be attached to help with assistant replies. E.g., for an API therapist attachments migh include previous diagnosis for the patient, as this is important context but something that would not have been on the public internet when GPT was trained.
2. *Code interpreter and functions* to allow it to perform actions, so you can build agents.
3. *JSON response* support, in case you want structured data back that you can use in a program.

Give it a whirl:
* Add Instructions (the equivalent of Context for chat)
* Upload a file document
* Ask it some questions (e.g., summarize this document)

## Usage
Go to the usage tab and have a look at your usage. This let's you send monthly spend caps and track your usage. I spend just pennies/month.


# Demo

Let's do a quick demo of a chat app I built recently to learn *react* programming (the *front end* of a web site).

# Build your own app

Next, let's get your own chat app working on top of the open AI APIs. Below is some boilerplate code that is easy to customize for simple chat completion. Run the cell now; it won't do anything, but we'll see how easy it is to build on top of in a minute.

When prompted, give the notebook permission to access the OPENAI_KEY secret.

In [2]:
!pip install openai
import openai
openai.api_key = userdata.get("OPENAI_KEY")
from google.colab import userdata

def chat (assistant_name : str, assistant_context : str, initial_message : str):
    """Start a chat settion with OpenAI's chatbot"""
    client_says = input(assistant_name + ": " + initial_message + " (type 'q' to end the session)\n").strip()
    messages = [
        {"role": "system", "content": assistant_context},
        {"role": "assistant", "content": initial_message},
        {"role": "user", "content": client_says},
    ]
    while client_says.lower() != "q":
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.2
        )
        assistant_says = response.choices[0].message.content
        print (f"{assistant_name}: {assistant_says}\n")
        messages.append({"role": "assistant", "content": assistant_says})
        client_says = input().strip()
        messages.append({"role": "user", "content": client_says})
    print ("Goodbye - I hope you enjoyed the sesion!")

# An AI therapist
def psycho_analyze():
    """Create a psycho-analysis chat assistant on top of OpenAI's chatbot"""
    chat ("AI therapist", "You are a helpful AI therapist who probes for more insights into your patient",
           "Hello, I am your AI therapist. Why have you come to see me today? ")

# An AI translator
def translate():
    """Create a translation chat assistant on top of OpenAI's chatbot"""
    target_language = input("What language do you want to translate to? ")
    chat ("Translator", f"Translate everything the user types into {target_language}",
          f"Type anything and I'll translate it into {target_language}")

In [4]:
# Now try running one of them, and experiment with building your own

translate()     # TODO - after trying this, comment out this line and uncomment one of the ones below

# psycho_analyze()

# chat (<assistant_name>, <assistant_context>, <initial_message>) # TODO - replace the placeholders with your own values



Translator: Je t'aime

Translator: Est-ce qu'il y a quelque chose d'autre avec lequel je peux t'aider?

Translator: N'hésitez pas à poser une question ou à demander de l'aide si vous en avez besoin.

Goodbye - I hope you enjoyed the sesion!


# DALL-E

Next, let's build our own version of DALL-E.


In [5]:
def create_image():
    """Create an image from a text prompt"""
    prompt = input("What do you want to see? ")
    size =   "1024x1024"  # Min size for DALL-E-3

    # Create an image URL from the prompt using the OpenAI API

    # Modify the call to openai.images.generate below to catch openai.BadRequestError and print the error code and message
    try:
        response = openai.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size=size
        )
    except openai.BadRequestError as e:
        print(f"Error: {e.code}. {e.body['message']}")
        return

    url = response.data[0].url
    print (f"Image created successfully at {url}")

create_image()

Error: content_policy_violation. Your request was rejected as a result of our safety system. Image descriptions generated from your prompt may contain text that is not allowed by our safety system. If you believe this was done in error, your request may succeed if retried, or by adjusting your prompt.


# Next up

Go back to the [OpenAI usage page](https://platform.openai.com/usage). You will see that you don't have much usage, even after playhing with it a bit. Image generation was the most expensive part.

Here are some resources to learn more:
* [OpenAI documentation](https://platform.openai.com/docs/overview)
* [OpenAI API guide](https://platform.openai.com/docs/api-reference)